# From the curriculum

In [1]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
%matplotlib inline

music = pd.DataFrame()
music['duration'] = [184, 134, 243, 186, 122, 197, 294, 382, 102, 264, 
                     205, 110, 307, 110, 397, 153, 190, 192, 210, 403,
                     164, 198, 204, 253, 234, 190, 182, 401, 376, 102]
music['loudness'] = [18, 34, 43, 36, 22, 9, 29, 22, 10, 24, 
                     20, 10, 17, 51, 7, 13, 19, 12, 21, 22,
                     16, 18, 4, 23, 34, 19, 14, 11, 37, 42]
music['bpm'] = [ 105, 90, 78, 75, 120, 110, 80, 100, 105, 60,
                  70, 105, 95, 70, 90, 105, 70, 75, 102, 100,
                  100, 95, 90, 80, 90, 80, 100, 105, 70, 65]

# Instructions

First let me state that these two models are fantastically awful. There doesn't seem to be much of a relationship. It's all very poor. However the increased variance in the weighted model is interesting.

Why don't you add the other feature and mess around with  kk  and weighting to see if you can do any better than we've done so far?

# My experiments

In [2]:
from sklearn import neighbors

## Your model here.
# Run the same model, this time with weights.
knn = neighbors.KNeighborsRegressor(n_neighbors=3)
knn_w = neighbors.KNeighborsRegressor(n_neighbors=3, weights='distance')
X = pd.DataFrame(music[["duration", "loudness"]])
Y = music.bpm
knn.fit(X, Y)
knn_w.fit(X, Y)

# Set up our prediction line.
T1 = np.arange(0, 50, 0.1)[:, np.newaxis]
T2 = np.arange(0, 50, 0.1)[:, np.newaxis]
T = np.concatenate((T1, T2), axis=1)

Y_ = knn_w.predict(T)

# plt.scatter(X, Y, c='k', label='data')
# plt.plot(T, Y_, c='g', label='prediction')
# plt.legend()
# plt.title('K=10, Weighted')
# plt.show()


from sklearn.model_selection import cross_val_score
score = cross_val_score(knn, X, Y, cv=5)
print("Unweighted Accuracy: %0.2f (+/- %0.2f)" % (score.mean(), score.std() * 2))
score_w = cross_val_score(knn_w, X, Y, cv=5)
print("Weighted Accuracy: %0.2f (+/- %0.2f)" % (score_w.mean(), score_w.std() * 2))

Unweighted Accuracy: -0.69 (+/- 1.03)
Weighted Accuracy: -0.59 (+/- 1.75)


In [28]:
from sklearn import neighbors
from sklearn.model_selection import cross_val_score

# set up data
X = pd.DataFrame(music[["duration", "loudness"]])
Y = music.bpm
T1 = np.arange(0, 50, 0.1)[:, np.newaxis]
T2 = np.arange(0, 50, 0.1)[:, np.newaxis]
T = np.concatenate((T1, T2), axis=1)

def testruns(n):
    for i in range(1,n):
        knn = neighbors.KNeighborsRegressor(n_neighbors=i)
        knn_w = neighbors.KNeighborsRegressor(n_neighbors=i, weights='distance')

        print("RESULTS FOR K = " + str(i))
        score = cross_val_score(knn, X, Y, cv=3)
        print("Unweighted Accuracy: %0.2f (+/- %0.2f)" % (score.mean(), score.std() * 2))
        score_w = cross_val_score(knn_w, X, Y, cv=3)
        print("Weighted Accuracy: %0.2f (+/- %0.2f)" % (score_w.mean(), score_w.std() * 2))
        
testruns(20)

RESULTS FOR K = 1
Unweighted Accuracy: -0.35 (+/- 0.62)
Weighted Accuracy: -0.35 (+/- 0.62)
RESULTS FOR K = 2
Unweighted Accuracy: -0.46 (+/- 0.50)
Weighted Accuracy: -0.19 (+/- 0.50)
RESULTS FOR K = 3
Unweighted Accuracy: -0.52 (+/- 0.45)
Weighted Accuracy: -0.19 (+/- 0.52)
RESULTS FOR K = 4
Unweighted Accuracy: -0.42 (+/- 0.17)
Weighted Accuracy: -0.11 (+/- 0.27)
RESULTS FOR K = 5
Unweighted Accuracy: -0.41 (+/- 0.49)
Weighted Accuracy: -0.09 (+/- 0.24)
RESULTS FOR K = 6
Unweighted Accuracy: -0.31 (+/- 0.38)
Weighted Accuracy: -0.06 (+/- 0.21)
RESULTS FOR K = 7
Unweighted Accuracy: -0.26 (+/- 0.19)
Weighted Accuracy: -0.03 (+/- 0.19)
RESULTS FOR K = 8
Unweighted Accuracy: -0.26 (+/- 0.23)
Weighted Accuracy: -0.03 (+/- 0.24)
RESULTS FOR K = 9
Unweighted Accuracy: -0.18 (+/- 0.21)
Weighted Accuracy: 0.00 (+/- 0.17)
RESULTS FOR K = 10
Unweighted Accuracy: -0.08 (+/- 0.18)
Weighted Accuracy: 0.02 (+/- 0.17)
RESULTS FOR K = 11
Unweighted Accuracy: -0.10 (+/- 0.07)
Weighted Accuracy: 0.02 

# Results:

Adding the second feature (duration) actually hurt the already terrible R-squared scores.  The peak score using both features was 0.04, barely distinguishable from a flat line across the mean.

An interesting result is that both accuracy scores improve with a higher K (until the sample size becomes too small to allow any higher).  My theory is that using more datapoints for each estimate, especially in a small dataset like this, helps combat overfitting.  This trend would persist for larger datasets, but more weakly, and topping out at a lower K-value.